# 🚀 Stable Diffusion WebSocket Server для Colab

**Запускає Flask сервер з WSS підтримкою через Cloudflare Tunnel**

Цей notebook:
- ✅ Встановлює Flask + Flask-SocketIO
- ✅ Запускає WebSocket сервер
- ✅ Встановлює Cloudflare Tunnel
- ✅ Генерує публічний WSS URL
- ✅ Приймає запити від WebUI
- ✅ Показує всі логи в реальному часі
- ✅ Монтує Google Drive для збереження

## 1️⃣ Встановлення залежностей

In [ ]:
import sys
print("📦 Встановлення залежностей...\n")

# Flask + WebSocket
!pip install -q flask flask-socketio python-socketio python-engineio
!pip install -q flask-cors

# PyTorch + CUDA
!pip install -q torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

# Diffusers
!pip install -q diffusers transformers accelerate safetensors
!pip install -q huggingface-hub omegaconf einops requests

# Image processing
!pip install -q pillow numpy opencv-python-headless

# Google Drive
!pip install -q google-auth-oauthlib google-auth-httplib2 google-api-python-client

print("\n✅ Все встановлено!")

## 2️⃣ Встановлення Cloudflare Tunnel (cloudflared)

In [ ]:
import os
import subprocess

print("📥 Завантажуємо Cloudflare Tunnel...")
!wget -q https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64
!chmod +x cloudflared-linux-amd64
!mv cloudflared-linux-amd64 cloudflared

print("✅ Cloudflare Tunnel встановлено!")
!./cloudflared --version

## 3️⃣ Монтування Google Drive

In [ ]:
from google.colab import drive
import os
from pathlib import Path

print("🔐 Монтування Google Drive...\n")
drive.mount('/content/drive')

# Створюємо папку для проекту
project_dir = Path('/content/drive/My Drive/StableDiffusion_Colab')
project_dir.mkdir(exist_ok=True)

# Папки для моделей та outputs
(project_dir / 'models' / 'checkpoints').mkdir(parents=True, exist_ok=True)
(project_dir / 'models' / 'loras').mkdir(parents=True, exist_ok=True)
(project_dir / 'outputs').mkdir(parents=True, exist_ok=True)

print(f"\n✅ Google Drive готов!")
print(f"📁 Папка проекту: {project_dir}")
print(f"📁 Models: {project_dir / 'models'}")
print(f"📁 Outputs: {project_dir / 'outputs'}")

## 4️⃣ Ініціалізація та підготовка

In [ ]:
import os
import sys
import json
import time
import torch
import threading
import subprocess
import base64
import traceback
from pathlib import Path
from datetime import datetime
from typing import Dict, List, Optional
import warnings
warnings.filterwarnings('ignore')

# Мультимедія
from PIL import Image
import io

# Flask + WebSocket
from flask import Flask, render_template, request, jsonify
from flask_socketio import SocketIO, emit, join_room, leave_room
from flask_cors import CORS

# Diffusers
from diffusers import StableDiffusionPipeline, StableDiffusionXLPipeline
from transformers import CLIPTokenizer
from huggingface_hub import hf_hub_download

# Google Drive
from pathlib import Path

# Setup
print(f"✅ PyTorch: {torch.__version__}")
print(f"✅ CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"✅ GPU: {torch.cuda.get_device_name(0)}")
    print(f"✅ GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")

# Конфігурація
HOST = '0.0.0.0'
PORT = 5000
DRIVE_PATH = Path('/content/drive/My Drive/StableDiffusion_Colab')

# Папки
models_dir = DRIVE_PATH / 'models'
outputs_dir = DRIVE_PATH / 'outputs'
checkpoints_dir = models_dir / 'checkpoints'
loras_dir = models_dir / 'loras'

checkpoints_dir.mkdir(parents=True, exist_ok=True)
loras_dir.mkdir(parents=True, exist_ok=True)
outputs_dir.mkdir(parents=True, exist_ok=True)

print(f"\n📁 Checkpoint dir: {checkpoints_dir}")
print(f"📁 LoRA dir: {loras_dir}")
print(f"📁 Output dir: {outputs_dir}")

# Глобальні змінні
pipe = None
current_model = None
hf_token = None
civitai_key = None
is_generating = False
is_downloading = False

print("\n✅ Ініціалізація завершена!")

## 5️⃣ Функція логування з реальним часом

In [ ]:
import sys
from datetime import datetime

class Logger:
    """Logger з реальним часом та розфарбуванням"""
    
    RESET = '\033[0m'
    BOLD = '\033[1m'
    RED = '\033[91m'
    GREEN = '\033[92m'
    YELLOW = '\033[93m'
    BLUE = '\033[94m'
    MAGENTA = '\033[95m'
    CYAN = '\033[96m'
    
    @staticmethod
    def log(message: str, level: str = 'INFO', prefix: str = ''):
        """Логування з часовою міткою"""
        timestamp = datetime.now().strftime('%H:%M:%S')
        
        if level == 'INFO':
            color = Logger.BLUE
            symbol = 'ℹ'
        elif level == 'SUCCESS':
            color = Logger.GREEN
            symbol = '✅'
        elif level == 'WARNING':
            color = Logger.YELLOW
            symbol = '⚠'
        elif level == 'ERROR':
            color = Logger.RED
            symbol = '❌'
        elif level == 'DOWNLOAD':
            color = Logger.CYAN
            symbol = '📥'
        elif level == 'GENERATE':
            color = Logger.MAGENTA
            symbol = '🎨'
        elif level == 'SERVER':
            color = Logger.GREEN
            symbol = '🚀'
        elif level == 'TUNNEL':
            color = Logger.CYAN
            symbol = '🌐'
        else:
            color = Logger.RESET
            symbol = '•'
        
        log_msg = f"{color}{symbol} [{timestamp}] {prefix}{message}{Logger.RESET}"
        print(log_msg, flush=True)
        sys.stdout.flush()

logger = Logger()
print("✅ Logger ініціалізовано!")

## 6️⃣ Створення Flask сервера з WebSocket та логуванням

In [ ]:
# Flask app
app = Flask(__name__)
app.config['SECRET_KEY'] = 'stable-diffusion-colab-secret'
CORS(app)

# WebSocket
socketio = SocketIO(app, cors_allowed_origins="*", async_mode='threading')

logger.log(f"Flask сервер готов на {HOST}:{PORT}", 'SERVER')

@app.route('/')
def index():
    return jsonify({"status": "connected", "message": "Stable Diffusion WebSocket Server"})

@socketio.on('connect')
def handle_connect():
    logger.log(f"Клієнт підключився: {request.sid}", 'INFO', 'CLIENT: ')
    emit('connection_response', {'status': 'connected', 'message': 'Server ready'})

@socketio.on('disconnect')
def handle_disconnect():
    logger.log(f"Клієнт відключився: {request.sid}", 'WARNING', 'CLIENT: ')

@socketio.on('set_api_keys')
def handle_api_keys(data):
    """Отримання API ключів від клієнта"""
    global hf_token, civitai_key
    hf_token = data.get('hf_token', '')
    civitai_key = data.get('civitai_key', '')
    logger.log("API ключі отримані", 'SUCCESS', 'API: ')
    emit('api_keys_saved', {'status': 'saved'}, broadcast=False)

@socketio.on('load_model')
def handle_load_model(data):
    """Завантажити модель"""
    global pipe, current_model
    model_id = data.get('model_id', 'runwayml/stable-diffusion-v1-5')
    
    try:
        logger.log(f"Завантажую модель: {model_id}", 'DOWNLOAD', 'MODEL: ')
        emit('status', {'message': f'Loading {model_id}...'})
        
        # Вибір класу залежно від моделі
        if 'xl' in model_id.lower():
            logger.log("Тип: SDXL", 'INFO', 'MODEL: ')
            pipe = StableDiffusionXLPipeline.from_pretrained(
                model_id,
                torch_dtype=torch.float16,
                use_safetensors=True,
                use_auth_token=hf_token
            )
        else:
            logger.log("Тип: SD", 'INFO', 'MODEL: ')
            pipe = StableDiffusionPipeline.from_pretrained(
                model_id,
                torch_dtype=torch.float16,
                use_safetensors=True,
                use_auth_token=hf_token
            )
        
        logger.log("Переношу на GPU...", 'DOWNLOAD', 'MODEL: ')
        pipe = pipe.to('cuda')
        pipe.enable_attention_slicing()
        
        current_model = model_id
        logger.log(f"Модель завантажена!", 'SUCCESS', 'MODEL: ')
        emit('model_loaded', {'model': model_id, 'status': 'ready'}, broadcast=True)
    
    except Exception as e:
        logger.log(f"Помилка: {str(e)}", 'ERROR', 'MODEL: ')
        logger.log(traceback.format_exc(), 'ERROR', 'TRACE: ')
        emit('error', {'message': str(e)}, broadcast=False)

@socketio.on('generate')
def handle_generate(data):
    """Генерувати зображення"""
    global pipe, is_generating
    
    if is_generating:
        logger.log("Вже генерую!", 'WARNING', 'GEN: ')
        emit('error', {'message': 'Already generating!'})
        return
    
    if not pipe:
        logger.log("Модель не завантажена!", 'ERROR', 'GEN: ')
        emit('error', {'message': 'Load a model first!'})
        return
    
    try:
        is_generating = True
        logger.log("Генерація почалася", 'GENERATE', 'GEN: ')
        emit('status', {'message': 'Generating...'})
        
        # Параметри
        prompt = data.get('prompt', '')
        negative_prompt = data.get('negative_prompt', '')
        height = data.get('height', 512)
        width = data.get('width', 512)
        steps = data.get('steps', 20)
        cfg = data.get('cfg_scale', 7.5)
        seed = data.get('seed', -1)
        
        logger.log(f"Prompt: {prompt[:60]}...", 'GENERATE', 'GEN: ')
        logger.log(f"Параметри: {width}x{height}, steps={steps}, cfg={cfg}", 'INFO', 'GEN: ')
        
        # Генератор
        if seed >= 0:
            generator = torch.Generator(device='cuda').manual_seed(seed)
            logger.log(f"Seed: {seed}", 'INFO', 'GEN: ')
        else:
            generator = None
            logger.log("Seed: random", 'INFO', 'GEN: ')
        
        start_time = time.time()
        
        # Генерація
        result = pipe(
            prompt,
            negative_prompt=negative_prompt,
            height=height,
            width=width,
            num_inference_steps=steps,
            guidance_scale=cfg,
            generator=generator
        )
        
        elapsed = time.time() - start_time
        logger.log(f"Готово за {elapsed:.2f}s", 'SUCCESS', 'GEN: ')
        
        # Конвертація у base64
        image = result.images[0]
        buffer = io.BytesIO()
        image.save(buffer, format='PNG')
        img_base64 = base64.b64encode(buffer.getvalue()).decode()
        
        # Збереження на Google Drive
        timestamp = int(time.time())
        output_path = outputs_dir / f"gen_{timestamp}.png"
        image.save(output_path)
        logger.log(f"Збережено: {output_path.name}", 'SUCCESS', 'SAVE: ')
        
        emit('image_generated', {
            'image': f'data:image/png;base64,{img_base64}',
            'seed': seed,
            'filename': output_path.name,
            'time': elapsed
        }, broadcast=True)
    
    except Exception as e:
        logger.log(f"Помилка: {str(e)}", 'ERROR', 'GEN: ')
        logger.log(traceback.format_exc(), 'ERROR', 'TRACE: ')
        emit('error', {'message': str(e)})
    
    finally:
        is_generating = False

logger.log("Event handlers зареєстровані", 'SUCCESS')

## 7️⃣ Запуск сервера та Cloudflare Tunnel з повним логуванням

In [ ]:
import time
import threading
import subprocess
import re
from IPython.display import HTML, display

# Запустити Flask сервер
def run_server():
    """Запустити Flask сервер"""
    logger.log(f"Запускаю на http://{HOST}:{PORT}", 'SERVER')
    socketio.run(app, host=HOST, port=PORT, debug=False, allow_unsafe_werkzeug=True)

# Запустити в фоновому потоці
logger.log("Запуск сервера...", 'SERVER')
server_thread = threading.Thread(target=run_server, daemon=True)
server_thread.start()

# Чекати запуск
logger.log("Чекаю запуск сервера...", 'INFO')
time.sleep(3)
logger.log("Сервер запущено!", 'SUCCESS', 'SERVER: ')

print("\n" + "="*60)
print("✅ СЕРВЕР ЗАПУЩЕНО!")
print("="*60 + "\n")

## 8️⃣ Запуск Cloudflare Tunnel з повним виводом логів

In [ ]:
import subprocess
import time
import re
import threading
from IPython.display import HTML, display

def run_tunnel():
    """Запустити Cloudflare Tunnel з повним логуванням"""
    logger.log("Запускаю Cloudflare Tunnel", 'TUNNEL')
    logger.log("Чекаю підключення (це може зайняти час)...", 'INFO', 'TUNNEL: ')
    print()
    
    process = subprocess.Popen(
        ['./cloudflared', 'tunnel', '--url', f'http://localhost:{PORT}'],
        stdout=subprocess.PIPE,
        stderr=subprocess.STDOUT,
        text=True,
        bufsize=1,
        universal_newlines=True
    )
    
    url_found = False
    tunnel_url = None
    
    try:
        for line in iter(process.stdout.readline, ''):
            line = line.strip()
            if line:
                # Показуємо всіх логи від cloudflared
                print(f"  🌐 {line}", flush=True)
                
            # Пошук URL
            if 'trycloudflare.com' in line and not url_found:
                match = re.search(r'https://[a-zA-Z0-9\-]+\.trycloudflare\.com', line)
                if match:
                    tunnel_url = match.group(0)
                    wss_url = tunnel_url.replace('https://', 'wss://')
                    
                    print("\n" + "="*70)
                    print("🎉 CLOUDFLARE TUNNEL ГОТІВ!")
                    print("="*70)
                    logger.log(f"Tunnel URL: {tunnel_url}", 'TUNNEL')
                    logger.log(f"WSS URL: {wss_url}", 'SUCCESS', 'TUNNEL: ')
                    print("="*70 + "\n")
                    
                    # HTML для легкого копіювання
                    html = f"""
                    <div style="border: 3px solid #00ff00; padding: 20px; border-radius: 10px; background: #1a1a1a; margin: 20px 0;">
                        <h2 style="color: #00ff00; margin: 0 0 15px 0; text-align: center;">✅ WSS URL ГОТІВ!</h2>
                        <hr style="border-color: #00ff00;">
                        <p style="margin: 10px 0; font-size: 14px; color: #0f0;">📍 Скопіюй цей URL:</p>
                        <input type="text" value="{wss_url}" 
                               style="width: 100%; padding: 12px; margin: 10px 0; font-family: monospace; font-size: 12px; 
                                      background:#2a2a2a; color:#0f0; border:2px solid #0f0; border-radius: 5px;" 
                               readonly onclick="this.select(); document.execCommand('copy');">
                        <p style="margin: 10px 0; font-size: 12px; color: #aaa;">👆 Клік щоб скопіювати</p>
                        <hr style="border-color: #00ff00;">
                        <h3 style="color: #0f0; margin: 15px 0 10px 0;">💡 Як використати:</h3>
                        <ol style="color: #aaa; margin: 10px 0;">
                            <li>Відкрий WebUI (index.html) в браузері</li>
                            <li>Settings tab → Server Configuration</li>
                            <li>Вставь URL в поле "WebSocket Server URL"</li>
                            <li>Натисни "Save URL"</li>
                            <li>F5 (Refresh сторінку)</li>
                            <li>✅ Готово! Генеруй зображення! 🎨</li>
                        </ol>
                    </div>
                    """
                    display(HTML(html))
                    url_found = True
    
    except KeyboardInterrupt:
        logger.log("Tunnel зупинено", 'WARNING', 'TUNNEL: ')
        process.kill()
    except Exception as e:
        logger.log(f"Помилка: {str(e)}", 'ERROR', 'TUNNEL: ')
        process.kill()

# Запустити tunnel в фоновому потоці
tunnel_thread = threading.Thread(target=run_tunnel, daemon=False)
tunnel_thread.start()

logger.log("Очікування на Tunnel...", 'INFO')
print()

## ✅ Готово! Сервер працює!

**Усі логи виводяться в реальному часі вище! ⬆️**

### 📋 Проверте вивід для:

1. **Встановлення залежностей** - Cell 1
2. **Cloudflare Tunnel** - Cell 2
3. **Google Drive монтування** - Cell 3
4. **Запуск сервера** - Cell 7
5. **WSS URL** - Cell 8 (зеленим)

### 🔗 Коли побачиш WSS URL (зелений):

1. **Скопіюй URL** з поля введення
2. **Відкрий WebUI** локально (index.html)
3. **Settings → Server Configuration**
4. **Вставь URL** в поле
5. **Save URL** → **F5 refresh**
6. **Генеруй зображення!** 🎨

### 📁 Google Drive:

- **Папка проекту**: `/My Drive/StableDiffusion_Colab`
- **Outputs**: `/My Drive/StableDiffusion_Colab/outputs`
- **Моделі**: `/My Drive/StableDiffusion_Colab/models`

### 📊 Логи показані для:

- ✅ Завантаження моделей
- ✅ Генерація зображень
- ✅ Підключення клієнтів
- ✅ Помилки
- ✅ Сервер (Flask + SocketIO)
- ✅ Tunnel (Cloudflare)

---

**Цей notebook буде працювати доки запущено Colab!** ⏱️

# 🚀 Stable Diffusion WebSocket Server для Colab

**Запускає Flask сервер з WSS підтримкою через Cloudflare Tunnel**

Цей notebook:
- ✅ Встановлює Flask + Flask-SocketIO
- ✅ Запускає WebSocket сервер
- ✅ Встановлює Cloudflare Tunnel
- ✅ Генерує публічний WSS URL
- ✅ Приймає запити від WebUI

**Результат:** Отримаєш `wss://` адресу для вставки в Settings → Server Configuration

## 1️⃣ Встановлення залежностей

In [ ]:
# Встановлення всіх залежностей
!pip install -q flask flask-socketio python-socketio python-engineio
!pip install -q torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!pip install -q diffusers transformers accelerate safetensors
!pip install -q huggingface-hub omegaconf einops requests
!pip install -q pillow numpy opencv-python-headless
!pip install -q flask-cors
!pip install -q pyngrok  # Для Cloudflare Tunnel альтернатива

print("✅ Все встановлено!")

## 2️⃣ Встановлення Cloudflare Tunnel (cloudflared)

In [ ]:
import os
import subprocess

# Завантажити cloudflared
print("📥 Завантажуємо Cloudflare Tunnel...")
!wget -q https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64
!chmod +x cloudflared-linux-amd64
!mv cloudflared-linux-amd64 cloudflared

print("✅ Cloudflare Tunnel встановлено!")
!./cloudflared --version

## 3️⃣ Ініціалізація та подготовка

In [ ]:
import os
import sys
import json
import time
import torch
import threading
import subprocess
import base64
from pathlib import Path
from datetime import datetime
from typing import Dict, List, Optional
import warnings
warnings.filterwarnings('ignore')

# Мультимедія
from PIL import Image
import io

# Flask + WebSocket
from flask import Flask, render_template, request, jsonify
from flask_socketio import SocketIO, emit, join_room, leave_room
from flask_cors import CORS

# Diffusers
from diffusers import StableDiffusionPipeline, StableDiffusionXLPipeline
from transformers import CLIPTokenizer
from huggingface_hub import hf_hub_download

# Setup
print(f"✅ PyTorch: {torch.__version__}")
print(f"✅ CUDA: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"✅ GPU: {torch.cuda.get_device_name(0)}")
    print(f"✅ Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")

# Папки
Path('models/checkpoints').mkdir(parents=True, exist_ok=True)
Path('models/loras').mkdir(parents=True, exist_ok=True)
Path('outputs').mkdir(parents=True, exist_ok=True)

print("\n✅ Папки створені!")

## 4️⃣ Створення Flask сервера з WebSocket

In [ ]:
# Конфігурація
HOST = '0.0.0.0'
PORT = 5000

# Глобальні змінні
pipe = None
current_model = None
hf_token = None
civitai_key = None
generation_queue = []
is_generating = False

# Flask app
app = Flask(__name__)
app.config['SECRET_KEY'] = 'stable-diffusion-colab-secret'
CORS(app)

# WebSocket
socketio = SocketIO(app, cors_allowed_origins="*", async_mode='threading')

print("✅ Flask сервер створено!")
print(f"✅ Буде запущено на: http://{HOST}:{PORT}")

## 5️⃣ WebSocket Event Handlers

In [ ]:
@app.route('/')
def index():
    return jsonify({"status": "connected", "message": "Stable Diffusion WebSocket Server"})

@socketio.on('connect')
def handle_connect():
    print(f"✅ Клієнт підключився: {request.sid}")
    emit('connection_response', {'status': 'connected', 'message': 'Server ready'})

@socketio.on('disconnect')
def handle_disconnect():
    print(f"❌ Клієнт відключився: {request.sid}")

@socketio.on('set_api_keys')
def handle_api_keys(data):
    """Отримання API ключів від клієнта"""
    global hf_token, civitai_key
    hf_token = data.get('hf_token', '')
    civitai_key = data.get('civitai_key', '')
    print(f"✅ API ключі отримані")
    emit('api_keys_saved', {'status': 'saved'}, broadcast=False)

@socketio.on('load_model')
def handle_load_model(data):
    """Завантажити модель"""
    global pipe, current_model
    model_id = data.get('model_id', 'runwayml/stable-diffusion-v1-5')
    
    try:
        print(f"🔄 Завантажую модель: {model_id}...")
        emit('status', {'message': f'Loading {model_id}...'})
        
        # Вибір класу залежно від моделі
        if 'xl' in model_id.lower():
            pipe = StableDiffusionXLPipeline.from_pretrained(
                model_id,
                torch_dtype=torch.float16,
                use_safetensors=True,
                use_auth_token=hf_token
            )
        else:
            pipe = StableDiffusionPipeline.from_pretrained(
                model_id,
                torch_dtype=torch.float16,
                use_safetensors=True,
                use_auth_token=hf_token
            )
        
        pipe = pipe.to('cuda')
        pipe.enable_attention_slicing()
        
        current_model = model_id
        print(f"✅ Модель завантажена!")
        emit('model_loaded', {'model': model_id, 'status': 'ready'}, broadcast=True)
    
    except Exception as e:
        print(f"❌ Помилка: {e}")
        emit('error', {'message': str(e)}, broadcast=False)

@socketio.on('generate')
def handle_generate(data):
    """Генерувати зображення"""
    global pipe, is_generating
    
    if is_generating:
        emit('error', {'message': 'Already generating!'})
        return
    
    if not pipe:
        emit('error', {'message': 'Load a model first!'})
        return
    
    try:
        is_generating = True
        emit('status', {'message': 'Generating...'})
        
        # Параметри
        prompt = data.get('prompt', '')
        negative_prompt = data.get('negative_prompt', '')
        height = data.get('height', 512)
        width = data.get('width', 512)
        steps = data.get('steps', 20)
        cfg = data.get('cfg_scale', 7.5)
        seed = data.get('seed', -1)
        
        # Генератор
        if seed >= 0:
            generator = torch.Generator(device='cuda').manual_seed(seed)
        else:
            generator = None
        
        print(f"🎨 Генерую: {prompt[:50]}...")
        
        # Генерація
        result = pipe(
            prompt,
            negative_prompt=negative_prompt,
            height=height,
            width=width,
            num_inference_steps=steps,
            guidance_scale=cfg,
            generator=generator
        )
        
        # Конвертація у base64
        image = result.images[0]
        buffer = io.BytesIO()
        image.save(buffer, format='PNG')
        img_base64 = base64.b64encode(buffer.getvalue()).decode()
        
        # Збереження
        output_path = Path('outputs') / f"gen_{int(time.time())}.png"
        image.save(output_path)
        
        print(f"✅ Готово! Збережено: {output_path.name}")
        emit('image_generated', {
            'image': f'data:image/png;base64,{img_base64}',
            'seed': seed,
            'filename': output_path.name
        }, broadcast=True)
    
    except Exception as e:
        print(f"❌ Помилка генерації: {e}")
        emit('error', {'message': str(e)})
    
    finally:
        is_generating = False

print("✅ Event handlers зареєстровані!")

## 6️⃣ Запуск сервера

In [ ]:
import time
import threading
import subprocess

def run_server():
    """Запустити Flask сервер"""
    print(f"\n🚀 Запускаю сервер на {HOST}:{PORT}...")
    socketio.run(app, host=HOST, port=PORT, debug=False, allow_unsafe_werkzeug=True)

# Запустити в фоновому потоці
server_thread = threading.Thread(target=run_server, daemon=True)
server_thread.start()

# Чекати запуск
print("⏳ Чекаю запуск сервера...")
time.sleep(3)
print("✅ Сервер запущено!")

## 7️⃣ Запуск Cloudflare Tunnel

In [ ]:
import subprocess
import time
import re
import threading
from IPython.display import HTML, display

def run_tunnel():
    """Запустити Cloudflare Tunnel"""
    print("🌐 Запускаю Cloudflare Tunnel...")
    print("⏳ Чекаю підключення (это займе деякий час)...\n")
    
    process = subprocess.Popen(
        ['./cloudflared', 'tunnel', '--url', f'http://localhost:{PORT}'],
        stdout=subprocess.PIPE,
        stderr=subprocess.PIPE,
        text=True,
        bufsize=1
    )
    
    url_found = False
    
    for line in process.stdout:
        print(line.strip())
        
        # Пошук URL
        if 'https://' in line and not url_found:
            match = re.search(r'https://[^\s]+', line)
            if match:
                tunnel_url = match.group(0)
                wss_url = tunnel_url.replace('https://', 'wss://')
                
                print(f"\n{'='*60}")
                print(f"🎉 CLOUDFLARE TUNNEL ГОТІВ!")
                print(f"{'='*60}")
                print(f"\n📍 СКОПІЮЙ ЦЕЙ URL:")
                print(f"\n   wss_URL (для WebUI):")
                print(f"   {wss_url}")
                print(f"\n   або як WebSocket:")
                print(f"   {tunnel_url.replace('https://', 'ws://')}")
                print(f"\n{'='*60}")
                print(f"\n💡 Вставити в WebUI:")
                print(f"   1. Відкрий index.html")
                print(f"   2. Settings → Server Configuration")
                print(f"   3. Вставь URL вище")
                print(f"   4. Натисни 'Save URL'")
                print(f"   5. F5 (refresh)")
                print(f"\n{'='*60}\n")
                
                # HTML виведення для простоти копіювання
                html = f"""
                <div style="border: 2px solid #00aa00; padding: 20px; border-radius: 10px; background: #1a1a1a; margin: 20px 0;">
                    <h2 style="color: #00aa00; margin: 0 0 15px 0;">✅ WSS URL Готів!</h2>
                    <p style="margin: 10px 0; font-size: 14px;">Скопіюй цей URL:</p>
                    <input type="text" value="{wss_url}" style="width: 100%; padding: 10px; margin: 10px 0; font-family: monospace; font-size: 12px;" readonly onclick="this.select();">
                    <p style="margin: 10px 0; font-size: 12px; color: #aaa;">Клік щоб виділити та скопіювати</p>
                </div>
                """
                display(HTML(html))
                url_found = True
    
    # Утримування процесу живим
    for line in process.stderr:
        print(line.strip())

# Запустити tunnel в фоновому потоці
tunnel_thread = threading.Thread(target=run_tunnel, daemon=True)
tunnel_thread.start()

print("⏳ Очікування на Tunnel URL...\n")

## ✅ Готово!

**Сервер запущено!** 🎉

Вище буде показаний `wss://` URL для вставки в WebUI Settings.

### Наступні кроки:

1. **Скопіюй WSS URL** з виводу вище
2. **Відкрий WebUI** (index.html в браузері)
3. **Settings → Server Configuration**
4. **Вставь URL** в поле "WebSocket Server URL"
5. **Натисни "Save URL"**
6. **F5 (Refresh)**
7. **Готово! Генеруй зображення!** 🎨

---

**Цей notebook буде працювати доки запущено Colab!** ⏱️